### location file format : map-making-app -> GeoGuessr

In [6]:
import json
from geopy.geocoders import Nominatim

In [12]:
def is_target_format(data):
    """
    入力データがすでに変換後のフォーマットに一致しているかを確認。
    """
    if isinstance(data, list):  # データがリストであることを確認
        for item in data:
            if not isinstance(item, dict) or "lat" not in item or "lng" not in item:
                return False
        return True
    return False

In [13]:
# 入力ファイルと出力ファイルのパス
input_file = './input/map-making-app_1.json'
output_file = './output/geoguessr_3.json'

# JSONファイルを読み込んで、customCoordinatesを抽出し、必要なら変換後フォーマットに変更
try:
    with open(input_file, 'r', encoding='utf-8') as infile:
        data = json.load(infile)
    
    if is_target_format(data):
        # 既に変換後のフォーマットの場合
        output_data = data
        print("入力データは既に変換フォーマットです。")
    else:
        # customCoordinatesを抽出
        custom_coordinates = data.get('customCoordinates', None)
        
        if custom_coordinates is None:
            print("customCoordinatesが見つかりませんでした。")
            output_data = []
        else:
            # customCoordinatesを出力データに設定
            output_data = custom_coordinates
            print("customCoordinatesを抽出しました。")
    
    # 結果を新しいJSONファイルに書き込む
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(output_data, outfile, ensure_ascii=False, indent=4)
    print(f"結果が {output_file} に保存されました。")

except FileNotFoundError:
    print(f"{input_file} が見つかりませんでした。")
except json.JSONDecodeError:
    print("JSONファイルの形式が正しくありません。")


customCoordinatesを抽出しました。
結果が ./output/geoguessr_3.json に保存されました。


#### 国名のタグ追加ver

In [4]:
# 緯度と経度から国名を返す
def get_country_name(latitude, longitude):
    geolocator = Nominatim(user_agent="abc")
    location = geolocator.reverse((latitude, longitude), language='jp')
    
    if location and 'country' in location.raw['address']:
        return location.raw['address']['country']
    else:
        return "unknown"

In [ ]:
# 入力ファイルと出力ファイルのパス
input_file = './input/map-making-app_1.json'
output_file = './output/geoguessr_3.json'

# JSONファイルを読み込んで、customCoordinatesを抽出し、必要なら変換後フォーマットに変更
try:
    with open(input_file, 'r', encoding='utf-8') as infile:
        data = json.load(infile)
    
    if is_target_format(data):
        # 既に変換後のフォーマットの場合
        output_data = data
        print("入力データは既に変換フォーマットです。")
    else:
        # customCoordinatesを抽出
        custom_coordinates = data.get('customCoordinates', None)
        
        if custom_coordinates is None:
            print("customCoordinatesが見つかりませんでした。")
            output_data = []
        else:
            # customCoordinatesを出力データに設定
            output_data = custom_coordinates
            print("customCoordinatesを抽出しました。")
    
    # 結果を新しいJSONファイルに書き込む
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(output_data, outfile, ensure_ascii=False, indent=4)
    print(f"結果が {output_file} に保存されました。")

except FileNotFoundError:
    print(f"{input_file} が見つかりませんでした。")
except json.JSONDecodeError:
    print("JSONファイルの形式が正しくありません。")


ue ついか
sita もと

In [7]:
# 入力ファイルと出力ファイルのパス
input_file = './input/geoguessr_1.json'
output_file = './output/geoguessr_2.json'



# JSONファイルを処理
try:
    with open(input_file, 'r', encoding='utf-8') as infile:
        data = json.load(infile)  # リスト形式のJSONを読み込む

    # 各要素にextra属性を追加
    for item in data:
        if isinstance(item, dict) and "lat" in item and "lng" in item:
            country_name = get_country_name(float(item["lat"]), float(item["lng"]))

            # 追加するextra属性のデータ
            item["extra"] = {
                "extra": {
                    "tags": [country_name],
                    "panoId": "NULL",
                    "panoDate": "NULL"
                }
            }

    # 結果を新しいJSONファイルに書き込む
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(data, outfile, ensure_ascii=False, indent=4)

    print(f"処理が完了しました。結果は {output_file} に保存されています。")
except FileNotFoundError:
    print(f"{input_file} が見つかりませんでした。")
except json.JSONDecodeError:
    print("JSONファイルの形式が正しくありません。")


処理が完了しました。結果は ./output/geoguessr_2.json に保存されています。


In [ ]:





import json

# 入力ファイルと出力ファイルのパス
input_file = 'input.json'
output_file = 'output.json'

# JSONファイルを処理
try:
    with open(input_file, 'r', encoding='utf-8') as infile:
        data = json.load(infile)  # リスト形式のJSONを読み込む

    # 各要素にextra属性を追加
    for item in data:
        if isinstance(item, dict) and "x" in item and "y" in item:
            # x と y の値を加算し文字列化
            sum_xy = str(item["x"] + item["y"])
            
            # extra属性を追加
            item["extra"] = {
                "tags": [sum_xy],
                "panoId": "MH3vmWH8YoiJCYQzTOQMMA",
                "panoDate": "2012-07"
            }

    # 結果を新しいJSONファイルに書き込む
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(data, outfile, ensure_ascii=False, indent=4)

    print(f"処理が完了しました。結果は {output_file} に保存されています。")
except FileNotFoundError:
    print(f"{input_file} が見つかりませんでした。")
except json.JSONDecodeError:
    print("JSONファイルの形式が正しくありません。")
